### 0.包导入和参数设置

In [ ]:
# 导入包
import argparse
import logging
import math
import os
import random
import shutil
from contextlib import nullcontext
from pathlib import Path
from collections import OrderedDict

import cv2
import datasets
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from datasets import load_dataset
from huggingface_hub import create_repo, upload_folder
from packaging import version
from peft import LoraConfig
from peft.utils import get_peft_model_state_dict
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
import torchvision.utils as tvu

import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, DDPMPipeline, DDIMPipeline, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import cast_training_params, compute_snr
from diffusers.utils import check_min_version, convert_state_dict_to_diffusers, is_wandb_available
from diffusers.utils.hub_utils import load_or_create_model_card, populate_model_card
from diffusers.utils.import_utils import is_xformers_available
from diffusers.utils.torch_utils import is_compiled_module

In [ ]:
# 设置参数
output_dir = './results/082801/'
logging_dir = 'logs'
pretrained_model_path = './pretrained_models/ddpm_church_256/'
mixed_precision = 'fp16'
report_to = 'wandb'
train_data_dir ='./data/exp/GaN/data/good/ab_free-png-g1-8400/' 
lr_scheduler = 'cosine'
prediction_type = None

gradient_accumulation_steps = 4
seed = 1337
rank = 4
learning_rate = 1e-4
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 1e-2
adam_epsilon = 1e-8
max_grad_norm = 1.0
max_train_steps = 12000
num_train_epochs = 100
lr_warmup_steps = 0
train_batch_size = 10
dataloader_num_workers = 4
noise_offset = 0
checkpointing_steps = 1000
resume_from_checkpoint = "latest"
checkpoints_total_limit = None
snr_gamma = None
max_train_samples = None

gradient_checkpointing = False
allow_tf32 = False
scale_lr = False
use_8bit_adam = False

In [ ]:
TrainConfig = dict(
    output_dir = output_dir,
    logging_dir = logging_dir,
    pretrained_model_path = pretrained_model_path,
    mixed_precision = mixed_precision,
    report_to = report_to,
    train_data_dir =train_data_dir,
    lr_scheduler = lr_scheduler,
    prediction_type = prediction_type,

    gradient_accumulation_steps = gradient_accumulation_steps,
    seed = seed,
    rank = rank,
    learning_rate = learning_rate,
    adam_beta1 = adam_beta1,
    adam_beta2 = adam_beta2,
    adam_weight_decay = adam_weight_decay,
    adam_epsilon = adam_epsilon, 
    max_grad_norm = max_grad_norm,
    max_train_steps = max_train_steps,
    num_train_epochs = num_train_epochs,
    lr_warmup_steps = lr_warmup_steps,
    train_batch_size = train_batch_size,
    dataloader_num_workers = dataloader_num_workers,
    noise_offset = noise_offset,
    checkpointing_steps = checkpointing_steps,
    resume_from_checkpoint = resume_from_checkpoint, 
    checkpoints_total_limit = checkpoints_total_limit,
    snr_gamma = snr_gamma,
    max_train_samples = max_train_samples,

    gradient_checkpointing = gradient_checkpointing,
    allow_tf32 = allow_tf32,
    scale_lr = scale_lr,
    use_8bit_adam = use_8bit_adam
)

### 1.配置日志和加速器

In [ ]:
logging_dir = Path(output_dir, logging_dir)

accelerator_project_config = ProjectConfiguration(project_dir=output_dir, logging_dir=logging_dir)

accelerator = Accelerator(
    gradient_accumulation_steps=gradient_accumulation_steps,
    mixed_precision=mixed_precision,
    log_with=report_to,
    project_config=accelerator_project_config,
)

### 2.硬件加速设置

In [ ]:
# Disable AMP for MPS.
if torch.backends.mps.is_available():
    accelerator.native_amp = False

### 3.检查并导入Wandb

In [ ]:
if report_to == "wandb":
    if not is_wandb_available():
        raise ImportError("Make sure to install wandb if you want to use it for logging during training.")
    import wandb

### 4.设置日志记录格式与详细程度

In [ ]:
logger = get_logger(__name__, log_level="INFO")

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)

if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_warning()
    diffusers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()
    diffusers.utils.logging.set_verbosity_error()

### 5.设置随机种子

In [ ]:
# If passed along, set the training seed now.
if seed is not None:
    set_seed(seed)

### 6.处理输出目录

In [ ]:
# Handle the repository creation
if accelerator.is_main_process:
    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)

### 7.模型组件加载

In [ ]:
# 加载预训练的DDPM模型
pipeline = DDPMPipeline.from_pretrained(pretrained_model_path)
# pipeline = DiffusionPipeline.from_pretrained(pretrained_model_path)

# 提取模型组件
noise_scheduler = pipeline.scheduler
unet = pipeline.unet

### 8.冻结模型参数，设置混合精度

In [ ]:
# freeze parameters of models to save more memory
unet.requires_grad_(False)

# For mixed precision training we cast all non-trainable weights (vae, non-lora text_encoder and non-lora unet) to half-precision
# as these weights are only used for inference, keeping weights in full precision is not required.
weight_dtype = torch.float32
if accelerator.mixed_precision == "fp16":
    weight_dtype = torch.float16
elif accelerator.mixed_precision == "bf16":
    weight_dtype = torch.bfloat16

# Freeze the unet parameters before adding adapters
for param in unet.parameters():
    param.requires_grad_(False)

In [ ]:
print(weight_dtype)

### 9.添加LoRA Adapter

In [ ]:
class LoRALayer(nn.Module):
    def __init__(self, original_layer, r, alpha):
        super(LoRALayer, self).__init__()
        self.original_layer = original_layer
        self.r = r
        self.alpha = alpha
        self.lora_up = nn.Linear(original_layer.in_features, r, bias=False)
        self.lora_down = nn.Linear(r, original_layer.out_features, bias=False)
        self.scaling = self.alpha / self.r

        # Initialize weights
        nn.init.kaiming_uniform_(self.lora_up.weight, a=math.sqrt(5))
        nn.init.zeros_(self.lora_down.weight)

    def forward(self, x):
        return self.original_layer(x) + self.lora_down(self.lora_up(x)) * self.scaling

def add_lora_adapters(unet, r, alpha):
    modules_to_modify = []
    for name, module in unet.named_modules():
        if isinstance(module, nn.Linear):
            modules_to_modify.append((name, module))
    
    for name, module in modules_to_modify:
        parent_module = unet
        for part in name.split('.')[:-1]:
            parent_module = getattr(parent_module, part)
        last_part = name.split('.')[-1]
        setattr(parent_module, last_part, LoRALayer(module, r, alpha))
    
    return unet

In [ ]:
# Add adapter and make sure the trainable params are in float32.
unet = add_lora_adapters(unet, rank, rank)

# Move unet, vae and text_encoder to device and cast to weight_dtype
unet.to(accelerator.device, dtype=weight_dtype)

if mixed_precision == "fp16":
    # only upcast trainable parameters (LoRA) into fp32
    cast_training_params(unet, dtype=torch.float32)

lora_layers = filter(lambda p: p.requires_grad, unet.parameters())

if gradient_checkpointing:
    unet.enable_gradient_checkpointing()

### 10.初始化优化器

In [ ]:
# Enable TF32 for faster training on Ampere GPUs,
# cf https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
if allow_tf32:
    torch.backends.cuda.matmul.allow_tf32 = True

if scale_lr:
    learning_rate = (
        learning_rate * gradient_accumulation_steps * train_batch_size * accelerator.num_processes
    )

# Initialize the optimizer
if use_8bit_adam:
    try:
        import bitsandbytes as bnb
    except ImportError:
        raise ImportError(
            "Please install bitsandbytes to use 8-bit Adam. You can do so by running `pip install bitsandbytes`"
        )

    optimizer_cls = bnb.optim.AdamW8bit
else:
    optimizer_cls = torch.optim.AdamW

optimizer = optimizer_cls(
    lora_layers,
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=adam_weight_decay,
    eps=adam_epsilon,
)

### 11.导入并预处理数据集

In [ ]:
if train_data_dir is not None:
    data_files = {"train": os.path.join(train_data_dir, "**")}
    dataset = load_dataset("imagefolder", data_files=data_files, cache_dir=None)
else:
    raise ValueError("Please provide a training data directory using --train_data_dir")

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

def unwrap_model(model):
        model = accelerator.unwrap_model(model)
        model = model._orig_mod if is_compiled_module(model) else model
        return model

def preprocess_train(examples):
    images = [image.convert("RGB") for image in examples["image"]]
    examples["pixel_values"] = [train_transforms(image) for image in images]
    return examples

### 12.创建DataLoader

In [ ]:
with accelerator.main_process_first():
    if max_train_samples is not None:
        dataset["train"] = dataset["train"].shuffle(seed).select(range(max_train_samples))
    # Set the training transforms
    train_dataset = dataset["train"].with_transform(preprocess_train)
# dataset = dataset.with_transform(preprocess_train)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=train_batch_size,
    num_workers=dataloader_num_workers,
    collate_fn=lambda examples: {
        "pixel_values": torch.stack([example["pixel_values"] for example in examples])
    }
)

### 13.创建Scheduler，计算训练步数

In [ ]:
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    lr_scheduler,
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps * accelerator.num_processes,
    num_training_steps=max_train_steps * accelerator.num_processes,
)



### 14.最后的准备与训练

In [ ]:
# Prepare everything with our `accelerator`.
unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    unet, optimizer, train_dataloader, lr_scheduler
)

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
if overrode_max_train_steps:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if accelerator.is_main_process:
    os.environ['WANDB_NOTEBOOK_NAME'] = 'Finetune.ipynb'
    # wandb.init()
    accelerator.init_trackers("DDPM-LoRA-fine-tune", config=TrainConfig)

# Train!
total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")
global_step = 0
first_epoch = 0

# initial_global_step = 0

# Potentially load in the weights and states from a previous save
if resume_from_checkpoint:
    if resume_from_checkpoint != "latest":
        path = os.path.basename(resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = os.listdir(output_dir)
        dirs = [d for d in dirs if d.startswith("checkpoint")]
        dirs = sorted(dirs, key=lambda x: int(x.split("-")[1]))
        path = dirs[-1] if len(dirs) > 0 else None

    if path is None:
        accelerator.print(
            f"Checkpoint '{resume_from_checkpoint}' does not exist. Starting a new training run."
        )
        resume_from_checkpoint = None
        initial_global_step = 0
    else:
        accelerator.print(f"Resuming from checkpoint {path}")

        accelerator.load_state(os.path.join(output_dir, path))
        global_step = int(path.split("-")[1])

        initial_global_step = global_step
        first_epoch = global_step // num_update_steps_per_epoch
else:
    initial_global_step = 0

progress_bar = tqdm(
    range(0, max_train_steps),
    initial=initial_global_step,
    desc="Steps",
    # Only show the progress bar once on each machine.
    disable=not accelerator.is_local_main_process,
)

for epoch in range(first_epoch, num_train_epochs):
    unet.train()
    train_loss = 0.0
    for step, batch in enumerate(train_dataloader):
        with accelerator.accumulate(unet):
            # 获取图像
            latents = batch["pixel_values"].to(accelerator.device, dtype=weight_dtype)
            
            # 生成噪声
            noise = torch.randn_like(latents)
            
            if noise_offset:
                # https://www.crosslabs.org//blog/diffusion-with-offset-noise
                noise += noise_offset * torch.randn(
                    (latents.shape[0], latents.shape[1], 1, 1), device=latents.device
                )

            bsz = latents.shape[0]
            # 随机选择时间步长
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
            timesteps = timesteps.long()

            # 添加噪声到潜变量
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            # Get the target for loss depending on the prediction type
            if prediction_type is not None:
                # set prediction_type of scheduler if defined
                noise_scheduler.register_to_config(prediction_type=prediction_type)

            if noise_scheduler.config.prediction_type == "epsilon":
                target = noise
            elif noise_scheduler.config.prediction_type == "v_prediction":
                target = noise_scheduler.get_velocity(latents, noise, timesteps)
            else:
                raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

            # 预测噪声并计算损失
            model_pred = unet(noisy_latents, timesteps, return_dict=False)[0]

            if snr_gamma is None:
                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
            else:
                # Compute loss-weights as per Section 3.4 of https://arxiv.org/abs/2303.09556.
                # Since we predict the noise instead of x_0, the original formulation is slightly changed.
                # This is discussed in Section 4.2 of the same paper.
                snr = compute_snr(noise_scheduler, timesteps)
                mse_loss_weights = torch.stack([snr, snr_gamma * torch.ones_like(timesteps)], dim=1).min(
                    dim=1
                )[0]
                if noise_scheduler.config.prediction_type == "epsilon":
                    mse_loss_weights = mse_loss_weights / snr
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    mse_loss_weights = mse_loss_weights / (snr + 1)

                loss = F.mse_loss(model_pred.float(), target.float(), reduction="none")
                loss = loss.mean(dim=list(range(1, len(loss.shape)))) * mse_loss_weights
                loss = loss.mean()

            avg_loss = accelerator.gather(loss.repeat(train_batch_size)).mean()
            train_loss += avg_loss.item() / gradient_accumulation_steps

            # 反向传播
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(unet.parameters(), max_grad_norm)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            progress_bar.update(1)
            global_step += 1
            accelerator.log({"train_loss": train_loss}, step=global_step)
            train_loss = 0.0

            if global_step % checkpointing_steps == 0:
                if accelerator.is_main_process:
                    # _before_ saving state, check if this save would set us over the `checkpoints_total_limit`
                    if checkpoints_total_limit is not None:
                        checkpoints = os.listdir(output_dir)
                        checkpoints = [d for d in checkpoints if d.startswith("checkpoint")]
                        checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))

                        # before we save the new checkpoint, we need to have at _most_ `checkpoints_total_limit - 1` checkpoints
                        if len(checkpoints) >= checkpoints_total_limit:
                            num_to_remove = len(checkpoints) - checkpoints_total_limit + 1
                            removing_checkpoints = checkpoints[0:num_to_remove]

                            logger.info(
                                f"{len(checkpoints)} checkpoints already exist, removing {len(removing_checkpoints)} checkpoints"
                            )
                            logger.info(f"removing checkpoints: {', '.join(removing_checkpoints)}")

                            for removing_checkpoint in removing_checkpoints:
                                removing_checkpoint = os.path.join(output_dir, removing_checkpoint)
                                shutil.rmtree(removing_checkpoint)

                    save_path = os.path.join(output_dir, f"checkpoint-{global_step}")
                    accelerator.save_state(save_path)

                    unwrapped_unet = unwrap_model(unet)
                    torch.save(unwrapped_unet.state_dict(), os.path.join(save_path, 'unet.ckpt'))

                    logger.info(f"Saved state to {save_path}")

        logs = {"step_loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
        progress_bar.set_postfix(**logs)

        if global_step >= max_train_steps:
            break